# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

### Student: Gerard Larkin
### Student Number: 20235986

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
lines = open("holbrook.txt").readlines()
data = []
# Write your code here
for line in lines:
    # create a dictionary with 3 keys: original, corrected, indexes
    dct = {}
    i = list(line.split())
    o_str = []
    for a in i:      
        if '|' in a:
            a1 = a.replace(a, a.split('|')[0])
            o_str.append(a1)
        else:
            a
            o_str.append(a)
    dct.update({'original': o_str})
    c_str = []
    for b in i:
        if '|' in b:
            b1 = b.replace(b, b.split('|')[1])
            c_str.append(b1)
        else:
            b
            c_str.append(b)
    dct.update({'corrected': c_str}) 
    i_str = ([c for c in range(len(o_str)) if o_str[c] != c_str[c]])
    dct.update({'indexes': i_str})
    
    data.append(dct)

    
print(data[2:5])
assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

[{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}, {'original': ['My', 'Dad', 'works', 'at', 'Melton', '.'], 'corrected': ['My', 'Dad', 'works', 'at', 'Melton', '.'], 'indexes': []}, {'original': ['My', 'siter', 'go', 'to', 'Tonbury', '.'], 'corrected': ['My', 'sister', 'goes', 'to', 'Tonbury', '.'], 'indexes': [1, 2]}]


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [2]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [3]:
from collections import Counter

correct_words = [i.lower() for j in [k['corrected'] for k in train] for i in j]

def unigram(word):
    # Write your code here.
    if isinstance(word, str):
        cnt = Counter()
        for i in correct_words:
            cnt[i] += 1
        dct = dict(cnt)     
        return dct[word]

    if isinstance(word, list):
        cnt = Counter()
        for i in correct_words:
            if i in word:
                cnt[i] += 1
        dct = dict(cnt)           
        return dct

def prob(word):
    # Write your code here.
    if isinstance(word, str):
        return unigram(word) / len(correct_words)
    
    word_prob = {}
    if isinstance(word, list):
        for k, v in unigram(word).items():      
            word_prob.update({k : v / len(correct_words)})    
        return word_prob
    
# Test your code with the following
assert(unigram("me")==87)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [4]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("sister", "popular"))

6


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [5]:
def get_candidates(token):
    dct = {}
    dist = []
    tok = [i.lower() for i in token]
    # creates a set of words in same order as list
    # http://www.martinbroadhurst.com/removing-duplicates-from-a-list-while-preserving-order-in-python.html
    words = set()
    corrected = [i for i in correct_words if not (i in words or words.add(i))] 
    for i in corrected:
        dct.update({i : edit_distance(tok, i)})
        dist.append(edit_distance(tok, i))
        
    return [i for i, j in dct.items() if j == min(dist)] # dict comprehension to return keys based on a particular value

# Test your code as follows
assert(get_candidates("minde") == ['mine', 'mind'])

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [6]:
def correct(sentence):
    # Write your code here
    corrected_sentence = []
    for i in sentence:
        if i in correct_words: # correct_words is list of corrected words in the training set 
            corrected_sentence.append(i)
        else:
            l = [i for i, j in prob(get_candidates(i)).items() if j == max(prob(get_candidates(i)).values())] # don't change
            # in case more than one word is returned from previous step, I will arbirtrarily choose the word that is the 
            # same length as the misspelled word (doesn't appear to be needed)
            if len(l) == 1:
                for k in l:
                    corrected_sentence.append(k)
            else:
                for k, v in prob(get_candidates(i)).items():
                    if len(k) == len(i):
                        corrected_sentence.append(k)
                
    return corrected_sentence

print(correct(["this","whitr","cat"]))
print("-------------------------")
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

['this', 'white', 'cat']
-------------------------


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [7]:
def accuracy(test):
    # Write your code here
    test_corr = [[j.lower() for j in i] for i in [k['corrected'] for k in test]]
    test_model_corr = []
    for i in range(len(test)):
        ol = test[i]['original']
        il = test[i]['indexes']
        rl = [ol[i] for i in il]
        rpl_l = correct(rl)
        for (i, rpl_l) in zip(il, rpl_l):
            ol[i] = rpl_l
        test_model_corr.append(ol)
    
    # flatten lists for comparison
    same_words = 0
    test_corr_flat = [i for j in test_corr for i in j]
    test_model_corr_flat = [i for j in test_model_corr for i in j]
    for i in test_corr_flat:
        if i in test_model_corr_flat:
            same_words += 1
    
    # accuracy of full corpus
    acc = "Accuracy Full Corpus: " + str(round(((same_words *100) / len(test_corr_flat)),2)) + "%"
        
    # accuracy of set corpus
    set_test_corr_flat = set(test_corr_flat)
    set_test_model_corr_flat = set(test_model_corr_flat)
    same_set_words = 0
    for i in set_test_corr_flat:
        if i in set_test_model_corr_flat:
            same_set_words += 1
    
    acc_set = "Accuracy Set Corpus: " + str(round(((same_set_words *100) / len(set_test_corr_flat)),2)) + "%"
    
    return acc, acc_set

print(accuracy(train))
# Results of running train data through model: ('Accuracy Full Corpus: 96.11%', 'Accuracy Set Corpus: 82.24%')
# Results of running test data through model: ('Accuracy Full Corpus: 86.63%', 'Accuracy Set Corpus: 73.2%')

('Accuracy Full Corpus: 96.11%', 'Accuracy Set Corpus: 82.24%')


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


## Description of implementation of improvement to corpus correction model

Our previous model was concerned with correcting spelling mistakes based on unigram probability(max probability of the word in the corpus and the minimum edit distance of altering one word to another) of a token in the corpus. This model was limited in that it only considered words in our small corpus and if a mis-spelled word did not have a corresponding word it could compare itself to it, e.g. 'consulatoin', it would ignore it.

We can imporve the accuracy of the model by increasing the list of words that we can compare our corpus against. This is done simply by calling the built-in nltk.corpus words.words().
However we need to mindful that every corpus has unique/specific words to that corpus that may not be in commom usage e.g. 'enflield'. We need to therefore ensure that this word is not corrected. I do this by setting a threshold limit of frequency of word the in the corpus being 2 or less when looking for words that are mis-spelled. If a word is not in the larger corpus and it only occurs once there is a high probability it is mis-spelled. Additonally, setting the threshold to 2 where the word is not in the larger corpus is indicative of the same spelling error occuring. I then run the word through a third party spellchecker module called 'autocorrect'.  

I then attempted to substitute bigrams with low proability in the small corpus with bigrams with a higher probability in the larger corpus based on the following:
* if the POS tag of the first token in the small corpus bigram tuple matches the POS tag in the first token of a large corpus bigram tuple
* and if the edit_distince of the second token in both bigrams was 1 at a maximum
* and the freq_dist (could not figure out how to calculate pmi, nor us the built-in pmi measure in nltk) of that token from the large corpus was the highest freq_dist out of all possible candidates

The large corpus of bigrams I will use is taken from https://norvig.com/ngrams/
The total number of bigrams in the this corpus is 314,843,401 as stated here https://ai.googleblog.com/2006/08/all-our-n-gram-are-belong-to-you.html

#### Result: 
I did not get the improvement on the model to work as intended. If I just keep to the additional spell checker part of the improved model the accuracy on the train data is actually less than when I originally ran the train data through the original model. I struggled to get the sought after bigram replacement after setting the conditions on which a bigram should be replaced.

 #### Correct data as per our original model

In [10]:
test_model_corr = []
orig_words = [[j.lower() for j in i] for i in [k['original'] for k in train]]
for i in range(len(orig_words)):
    ol = orig_words[i]
    il = train[i]['indexes']
    rl = [ol[i] for i in il]
    rpl_l = correct(rl) # first correct words as per the previously built model
    for (i, rpl_l) in zip(il, rpl_l):
        ol[i] = rpl_l
    test_model_corr.append(ol)      

#### Get frequency of each word in the corpus

In [11]:
mod_corr_words = [i for j in test_model_corr for i in j]
word_freq = dict()
for word in mod_corr_words:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1

#### Correct spelling using autocorrect: https://github.com/fsondej/autocorrect

In [12]:
from nltk.corpus import words
from autocorrect import Speller
spell = Speller()
spell_test_model_corr = []
for i in test_model_corr:
    sub_list = []
    for j in i:
        # choosing a freq threshold of 2, if the freq of a word is 1 and it is not in words the most likely spelling error, additionally 
        # spelling errors are repeated on the same word, but want to leave the threshold low so that I don;t include corpus specific words
        if word_freq[j] <= 2 and j not in words.words():
            k = spell(j)
            sub_list.append(k)
        else:
            sub_list.append(j)
    spell_test_model_corr.append(sub_list)

#### Replace n-grams (bigram) in corpus with low probability of occuring with n-gram with higher probability in larger corpus

In [14]:
import nltk
from nltk.collocations import *
from nltk.corpus import brown
import collections

bigram_measures = nltk.collocations.BigramAssocMeasures()

text = open("ngrams/count_2w.txt").readlines()
big_corpus_bg_score = [] #need
for i in text:
    i = i.replace("\t"," ").replace("\n","")
    i = i.split()
    i[2] = int(i[2])/314843401
    big_corpus_bg_score.append(i)
    
corpus_bg = BigramCollocationFinder.from_words([i.lower() for j in spell_test_model_corr for i in j])

corpus_word_fd = nltk.FreqDist([i.lower() for j in spell_test_model_corr for i in j])
corpus_bigram_fd = nltk.FreqDist(nltk.bigrams([i.lower() for j in spell_test_model_corr for i in j]))

corpus_bg = BigramCollocationFinder(corpus_word_fd, corpus_bigram_fd)
score_corpus_bg = corpus_bg.score_ngrams(bigram_measures.raw_freq) 

lst1 = [list(i[0]) for i in score_corpus_bg]
lst2 = [i[1] for i in score_corpus_bg]
corpus_bg_score = [[j, *k, i] for i,(j, *k) in zip(lst2, lst1)] #maybe need

corpus_bg_tag_0 = nltk.pos_tag([x[0] for x in corpus_bg_score]) # ('9.30', 'CD'), ('abc', 'NN')
corpus_bg_tag_1 = nltk.pos_tag([x[1] for x in corpus_bg_score])

corpus_bg_tags = [(i,j) for i, j in zip(corpus_bg_tag_0, corpus_bg_tag_1)] # (('9.30', 'CD'), ('a.m.', 'RB')), (('abc', 'NN'), ('crazy', 'JJ')) #need

big_corpus_bg_tag_0 = nltk.pos_tag([x[0] for x in big_corpus_bg_score])
big_corpus_bg_tag_1 = nltk.pos_tag([x[1] for x in big_corpus_bg_score])

big_corpus_bg_tags = [(i,j) for i, j in zip(big_corpus_bg_tag_0, big_corpus_bg_tag_1)] #need

In [20]:
bigram_spell_test_model_corr = []
for i, j in zip(corpus_bg_score, big_corpus_bg_score):
    # check if full bigram with tags match to bigram from larger corpus, if so add back as most likely correct
    if i == j: 
        k = j[0][0],j[1][0]
        bigram_spell_test_model_corr.append(k)
    # check if first element of the bigram matches and if tag of first element matches and the second bigram element does 
    # not match and the tag of second element does not match and if edit_distance of second tag = 1 and then take result with highest frequency
    # get max freq bigram from a list of potential candidate bigrams 
    #max_freq_bg = [j for j in big_corpus_bg_score if j[0][1] == max(j[0][1])]
    if i[0][0] == j[0][0] and i[0][1] == j[0][1] and \
        i[1][0] != j[1][0] and i[1][1] != j[1][1] and \
        edit_distance(i[1][0],j[1][0]) == 1:
            k = j[0][0],j[1][0]
            bigram_spell_test_model_corr.append(k)

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [18]:
same_words = 0
corr_flat_list = correct_words
model_corr_flat_list = [i for j in spell_test_model_corr for i in j]
for i, j in zip(corr_flat_list, model_corr_flat_list):
    if i == j:
        same_words += 1

set_same_words = 0
set_corr_flat_list = set(corr_flat_list)
set_model_corr_flat_list = set(model_corr_flat_list)
for i, j in zip(set_corr_flat_list, set_model_corr_flat_list):
    if i == j:
        set_same_words += 1

# accuracy of full corpus
print("Accuracy Full Corpus: " + str(round(((same_words) / len(corr_flat_list)) * 100,2)) + "%") # Accuracy Full Corpus: 91.58%
# accuracy of set corpus
print("Accuracy Set Corpus: " + str(round(((set_same_words) / len(set_corr_flat_list)) * 100 ,2)) + "%") # Accuracy Set Corpus: 87.67%

Accuracy Full Corpus: 91.6%
Accuracy Set Corpus: 0.33%
